# CSE 144 Spring 2023 Assignment 1

This course's initial assignment comprises 10 questions, each carrying a score of 6, amounting to a total score of 60. 

In this exciting assignment, you'll get the chance to train a logistic regression model and your very own neural network on a real-world dataset! But that's not all – you'll also practice your skills in interacting with Kaggle, the leading platform for machine learning competitions. Get ready to dive in and unleash your data science skills!

In [145]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format="retina"
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import random
import torch
from torch import nn, optim
import math
from IPython import display

# Data preparation

In [146]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Dataset: Customer Churn Prediction 2020**

To begin working on this code, it is necessary to obtain the Customer Churn Prediction 2020 dataset, which can be downloaded from the [following page](https://www.kaggle.com/competitions/customer-churn-prediction-2020/overview). After downloading the dataset, you can proceed to upload the train.csv and test.csv files to Colab for accessing them in your notebook.

In [147]:
train_df = pd.read_csv("/content/drive/MyDrive/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/test.csv")

In [148]:
train_df.head()

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,OH,107,area_code_415,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,no
1,NJ,137,area_code_415,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,no
2,OH,84,area_code_408,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,no
3,OK,75,area_code_415,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,no
4,MA,121,area_code_510,no,yes,24,218.2,88,37.09,348.5,108,29.62,212.6,118,9.57,7.5,7,2.03,3,no


In [149]:
train_df.describe()

,account_length,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
count,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000
mean,100.236235,7.631765,180.259600,99.907294,30.644682,200.173906,100.176471,17.015012,200.527882,99.839529,9.023892,10.256071,4.426353,2.769654,1.559059
std,39.698401,13.439882,54.012373,19.850817,9.182096,50.249518,19.908591,4.271212,50.353548,20.093220,2.265922,2.760102,2.463069,0.745204,1.311434
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,73.000000,0.000000,143.325000,87.000000,24.365000,165.925000,87.000000,14.102500,167.225000,86.000000,7.522500,8.500000,3.000000,2.300000,1.000000
50%,100.000000,0.000000,180.450000,100.000000,30.680000,200.700000,100.000000,17.060000,200.450000,100.000000,9.020000,10.300000,4.000000,2.780000,1.000000
75%,127.000000,16.000000,216.200000,113.000000,36.750000,233.775000,114.000000,19.867500,234.700000,113.000000,10.560000,12.000000,6.000000,3.240000,2.000000
max,243.000000,52.000000,351.500000,165.000000,59.760000,359.300000,170.000000,30.540000,395.000000,175.000000,17.770000,20.000000,20.000000,5.400000,9.000000


**Setting a Seed for Random Number Generators in Python and PyTorch**

In [150]:
seed = 1
random.seed(seed)
torch.manual_seed(seed)

**Shuffle the training data**

In [151]:
# Show first five elements in the input
def show_first_five_elements(input):
  return list(input)[:5]

train_df_index = train_df.index
print(f"Initial index of train_df: {show_first_five_elements(train_df_index)}")

shuffled_index = np.random.permutation(train_df.index)
print(f"Shuffled index: {show_first_five_elements(shuffled_index)}")

train_df = train_df.reindex(shuffled_index)
print(f"Shuffled index of train_df: {show_first_five_elements(train_df.index)}")

print("\nThe examples in train_df are shuffled:")
train_df.head()

Initial index of train_df: [0, 1, 2, 3, 4]
Shuffled index: [1834, 604, 314, 3166, 3337]
Shuffled index of train_df: [1834, 604, 314, 3166, 3337]

The examples in train_df are shuffled:


,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
1834,VA,75,area_code_415,no,no,0,224.7,116,38.20,192.0,79,16.32,212.2,98,9.55,11.3,11,3.05,3,no
604,OR,129,area_code_510,no,yes,36,192.8,103,32.78,177.0,83,15.05,216.5,118,9.74,16.4,5,4.43,1,no
314,KS,127,area_code_415,no,no,0,221.0,100,37.57,160.7,113,13.66,233.1,96,10.49,6.8,4,1.84,2,no
3166,MN,113,area_code_408,no,no,0,198.6,115,33.76,258.0,106,21.93,245.8,104,11.06,10.2,6,2.75,1,yes
3337,MA,152,area_code_408,no,no,0,216.7,70,36.84,152.7,102,12.98,181.2,99,8.15,11.6,6,3.13,1,no


**Selecting features for training and testing**

In [152]:
X = train_df[['state', 'account_length', 'area_code', 'international_plan',
       'voice_mail_plan', 'number_vmail_messages', 'total_day_minutes',
       'total_day_calls', 'total_day_charge', 'total_eve_minutes',
       'total_eve_calls', 'total_eve_charge', 'total_night_minutes',
       'total_night_calls', 'total_night_charge', 'total_intl_minutes',
       'total_intl_calls', 'total_intl_charge',
       'number_customer_service_calls']]
Y = train_df['churn']

X_test = test_df[['state', 'account_length', 'area_code', 'international_plan',
       'voice_mail_plan', 'number_vmail_messages', 'total_day_minutes',
       'total_day_calls', 'total_day_charge', 'total_eve_minutes',
       'total_eve_calls', 'total_eve_charge', 'total_night_minutes',
       'total_night_calls', 'total_night_charge', 'total_intl_minutes',
       'total_intl_calls', 'total_intl_charge',
       'number_customer_service_calls']]

test_id = test_df['id']


**Check missing data in the train and test datasets**

In [153]:
print(f"Count of missing values in the train_df: {train_df.isnull().sum().sum()}")
print(f"Count of missing values in the test_df: {test_df.isnull().sum().sum()}")

Count of missing values in the train_df: 0
Count of missing values in the test_df: 0


**Replace 'no' with 0 and 'yes' with 1 in the Y DataFrame**

Question 1: In order to prepare Y for future training purposes, consider converting the current string values of 'no' and 'yes' into binary integers, 0 and 1, respectively. Hint: This can be accomplished by using the replace function.

In [154]:
###### Your codes start here.######
Y = Y.replace("no", "0")
Y = Y.replace("yes", "1")
###### Your codes end here.######

# Convert Y DataFrame to integer type
Y = Y.astype(int)

Question 2: Split X and Y into training set & validation set. Hint: try train_test_split function.

In [155]:
###### Your codes start here.######
X_train, X_val, y_train, y_val = train_test_split(X, Y) 

###### Your codes end here.######

In [156]:
X_train.head()

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
1121,VA,118,area_code_415,no,no,0,154.6,112,26.28,184.2,105,15.66,217.4,102,9.78,12.6,5,3.40,2
3265,MA,83,area_code_408,no,no,0,286.6,114,48.72,101.0,138,8.59,199.6,146,8.98,13.2,4,3.56,1
1113,MT,10,area_code_510,no,no,0,183.0,103,31.11,214.8,77,18.26,206.4,73,9.29,8.7,6,2.35,2
3374,ID,95,area_code_408,no,no,0,133.0,94,22.61,196.6,126,16.71,181.0,128,8.15,9.9,3,2.67,4
2730,SC,88,area_code_408,no,no,0,153.5,94,26.10,251.7,118,21.39,182.2,99,8.20,8.5,6,2.30,1


**Convert categorical features in a DataFrame to one-hot encoding**

In [157]:
def convert_features_to_one_hot(df, feature_name_list):
  for feature_name in feature_name_list:
    df = pd.get_dummies(df, columns=[feature_name])
  
  return df

Question 3: Create a list that contains all categorical features in the dataset, then use function convert_features_to_one_hot to transform the categorical features into one-hot encoded representations.

In [158]:
# List of categorical features to be one-hot encoded
###### Your codes start here.#####
#y_train = train_df['churn']
features = ['state', 'area_code', 'international_plan',
       'voice_mail_plan']

X_train = convert_features_to_one_hot(X_train, features)
X_val = convert_features_to_one_hot(X_val, features)
X_test = convert_features_to_one_hot(X_test, features)

###### Your codes end here.######

In [159]:
X_train.head()

,account_length,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,...,state_WI,state_WV,state_WY,area_code_area_code_408,area_code_area_code_415,area_code_area_code_510,international_plan_no,international_plan_yes,voice_mail_plan_no,voice_mail_plan_yes
1121,118,0,154.6,112,26.28,184.2,105,15.66,217.4,102,...,0,0,0,0,1,0,1,0,1,0
3265,83,0,286.6,114,48.72,101.0,138,8.59,199.6,146,...,0,0,0,1,0,0,1,0,1,0
1113,10,0,183.0,103,31.11,214.8,77,18.26,206.4,73,...,0,0,0,0,0,1,1,0,1,0
3374,95,0,133.0,94,22.61,196.6,126,16.71,181.0,128,...,0,0,0,1,0,0,1,0,1,0
2730,88,0,153.5,94,26.10,251.7,118,21.39,182.2,99,...,0,0,0,1,0,0,1,0,1,0


**Save Predicted Churn Values to CSV File**

**Load train and test datasets from CSV files**

In [160]:
def save_prediction_to_csv_file(y_test_pred, file_name):
  # Convert predicted churn values to 'yes' and 'no'
  churn_list = ['no' if pred == 0 else 'yes' for pred in y_test_pred]
  
  # Create a DataFrame containing the IDs and predicted churn values
  submission_df = pd.DataFrame({'id': test_df['id'], 'churn': churn_list})

  # Save the DataFrame to a CSV file
  submission_df.to_csv(file_name, index=False)

# Logistic regression

**Fit logistic regression model on training data**

Question 4: Train a logistic regression model using X_train and Y_train with l2 penalty. Hint: Check LogisticRegression from sklearn.

In [161]:
###### Your codes start here.######

LR_model = LogisticRegression(penalty='l2', max_iter=5800)
LR_model.fit(X_train, y_train)

###### Your codes end here.######

LogisticRegression(max_iter=5800)

**Make predictions on validation data and print validation accuracy**

Question 5: Generate predictions on the validation dataset using the predict function, and subsequently, assess the model's predictive performance by comparing the predicted results against the true labels, computing the accuracy metric as a measure of performance.

In [162]:
###### Your codes start here.######
y_pred = LR_model.predict(X_val)
accuracy = LR_model.score(X_val, y_val)
print(accuracy)
###### Your codes end here.######

0.8664158043273753


**Make predictions on test data and save to CSV file**

In [163]:
y_test_pred = LR_model.predict(X_test)
save_prediction_to_csv_file(y_test_pred, "submission_lr.csv")

After obtaining the submission file, submission_lr.csv, proceed to upload it to the designated competition page on [kaggle](https://www.kaggle.com/competitions/customer-churn-prediction-2020/submissions). This will allow you to assess the performance of your model on the competition's test set by viewing the corresponding evaluation results.

# Neural Network

**Set device to GPU if available, otherwise use CPU**

Question 6: Prior to executing the code, verify if a GPU is available and if so, assign it to a variable named device. Otherwise, utilize the CPU for computation. This can be accomplished by using the torch.device function.

In [164]:
###### Your codes start here.######
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
###### Your codes end here.######

**Convert data to PyTorch tensors and move to device**

In [165]:
# Convert training data to PyTorch tensors and move to device
X_train = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train.values, dtype=torch.long).to(device)

# Convert validation data to PyTorch tensors and move to device
X_val = torch.tensor(X_val.values, dtype=torch.float32).to(device)
y_val = torch.tensor(y_val.values, dtype=torch.long).to(device)

# Convert test data to PyTorch tensors and move to device
X_test = torch.tensor(X_test.values, dtype=torch.float32).to(device)

**Define network architecture and hyperparameters**

In [166]:
D = X_train.shape[1]  # feature dimensions
C = 2  # num_classes
H = 40  # num_hidden_units

# Set learning rate and regularization strength
learning_rate = 1e-3
lambda_l2 = 1e-3

# Set number of training epochs
epochs = 27000

**Train a Linear Model with PyTorch**

Question 7: 

For each question, there are 2 scores available.

1. Construct your initial neural network architecture, consisting of three fully-connected layers with ReLU activation functions inserted between each pair of fully-connected layers. The network should consist of D input feature dimensions, H hidden units in each layer, and C output classes. Hint: Use nn.Sequential function.

2. Implement the cross-entropy loss function to calculate the loss between predicted and ground-truth labels. Hint: use the CrossEntropyLoss function provided by PyTorch.

3. Use the Adam optimizer to update the model's parameters during training. Set the weight decay parameter to lambda_l2.

Notice: Since CrossEntropyLoss function already includes a sigmoid activation function, there's no need to include an additional sigmoid layer in your model. The CrossEntropyLoss function combines the sigmoid activation and the binary cross-entropy loss function into a single function.

In [167]:
###### 1. Your codes start here.######
class NeuralNet(nn.Module):
  def __init__(self, D, H, C):
    super(NeuralNet, self).__init__()
    # defining layers and applying reLU in between
    # hidden units receives input from previous layer
    # applies linear function to it and then a nonlinear (reLU)
    self.layer1 = nn.Linear(D, H)
    self.r1 = nn.ReLU()
    self.layer2 = nn.Linear(H, H)
    self.r2 = nn.ReLU()
    self.layer3 = nn.Linear(H, C)

  def forward(self, x):
    # this function is specfically for x and how it's
    # passed thru the layers
     x = self.layer1(x)
     x = self.r1(x)
     x = self.layer2(x)
     x = self.r2(x)
     x = self.layer3(x)
     return x

model = NeuralNet(D, H, C)
###### Your codes end here.######
model.to(device)

# nn package has different loss functions.
# we use cross entropy loss for our classification task
###### 2. Your codes start here.######

loss_function = nn.CrossEntropyLoss()

###### Your codes end here.######

# we use the optim package to apply
# ADAM for our parameter updates
###### 3. Your codes start here.######

adam_optim = optim.Adam(model.parameters(), weight_decay=lambda_l2)
###### Your codes end here.######

Question 8: Train your neural network model using a for loop. For guidance on how to write the training code, refer to [this link](https://github.com/yizuc/CSE144/blob/main/05-regression.ipynb) for an example implementation.

In [168]:
# Training
for t in range(epochs):
    ###### Your codes start here.######
    # feed forward
    y_pred = model(X_train)

    # compute loss
    loss = loss_function(y_pred, y_train)

    # zeros gradient before running
    # backwards pass
    adam_optim.zero_grad()

    # backward pass
    loss.backward()

    # update params
    adam_optim.step()

# calculating and printing out accuracy
# axis 1 shows the predicted values, class probabilities
predicted = torch.argmax(y_pred, axis=1)
# this line determines how many predictions were right
correct = (predicted == y_train).sum().item()
# total number of samples in y_train
total = y_train.size(0)
accuracy = correct/total

print(accuracy)
    
    ###### Your codes end here.######

0.972074050831503


**Evaluate neural network model on validation data**

Question 9: Explain the purpose of using the with torch.no_grad() context manager in the following code implementation.

In [169]:
with torch.no_grad():
  y_val_pred = model(X_val)
  _, predicted_val = torch.max(y_val_pred, 1)
  val_acc = (y_val == predicted_val).sum().float() / len(y_val)

  # Print validation accuracy
  print(f"Neural network model validation accuracy: {val_acc}")

Neural network model validation accuracy: 0.866415798664093


ANSWER: torch.no_grad() is used to disable gradient calculation. After reading about this function, I found that it is used in situations where you don't need to calculate a gradient, like evaluating a pre-trained model. We are doing exactly that! Evaluating a pre-trained model. This is beneficial because it speeds up testing and reduces memory use.

**Evaluate neural network model on test data**

In [170]:
with torch.no_grad():
  y_test_pred = model(X_test)
  _, predicted_test = torch.max(y_test_pred, 1)

**Save predicted test values to CSV file**

In [171]:
save_prediction_to_csv_file(predicted_test, "submission_nn.csv")

After generating the submission_nn.csv file, proceed to upload it to the designated competition page on [kaggle](https://www.kaggle.com/competitions/customer-churn-prediction-2020/submissions) to assess the performance of the neural network model. You can then compare the neural network's performance against the logistic regression model and determine whether it outperforms it.

Question 10: Adjust the value of H in your neural network architecture from 6 to 40, and retrain the model. Then, answer the following questions, each of which carries a score of 2:

1. How does the training accuracy change after increasing H?
2. How does the validation accuracy change after increasing H?
3. What is the likely explanation for the observed changes in validation accuracy after increasing H?

1. The training accuracy went up. It started at apx. 92 and is now at 99.
2. The validation accuracy dropped. From apx. 91 to 85.
3. Increasing the number of hidden units in a neural network means it can capture more patterns and pick up more in the training dataset, which explains the increase in the training accuracy. On the other hand, this can make the model more complicated, which can lead to overfitting– Because of the many layers, the model may begin to memorize the training data instead of learning generalized patterns. This means it may not perform well on new data. This is why the validation accuracy decreased.
